#Implementação do algoritmo para calcular r pelo método de Newton original.

In [1]:
import numpy as np
import time
import pandas as pd

In [2]:
# Definição da função f(r) e sua derivada analítica f'(r)
def f(r, nu, mu):
    return nu * r**3 - 9 * mu * r + 3

def f_prime(r, nu, mu):
    return 3 * nu * r**2 - 9 * mu


In [20]:
# Método de Newton Original
def newton_original(x0, nu, mu, tol=1e-3, max_iter=100):
    x = x0
    for i in range(max_iter):
        fx = f(x, nu, mu)
        fpx = f_prime(x, nu, mu)
        if fpx == 0:  # Evita divisão por zero
            print("Derivada zero. Método não converge.")
            return None, None
        x_next = x - fx / fpx
        if abs(x_next - x) < tol:  # Critério de convergência
            return x_next, i + 1
        x = x_next
    print("Máximo de iterações atingido.")
    return x, max_iter

# Implementção do algoritmo para calcular r pelo método de Newton com FL.

In [19]:
# Método de Newton com Fator de Limitação (FL)
def newton_com_fl(x0, nu, mu, lam, tol=1e-3, max_iter=100):
    x = x0
    for i in range(max_iter):
        fx = f(x, nu, mu)
        fpx = derivada_numerica(f, x, nu, mu)  # Usando derivada simples

        # Cálculo do fator FL
        FL = fpx if abs(fpx) >= lam else lam

        x_next = x - fx / FL
        if abs(x_next - x) < tol:  # Critério de convergência
            return x_next, i + 1
        x = x_next
    print("Máximo de iterações atingido.")
    return x, max_iter

#Implementando método numérico para achar derivada de f(r) e refazer item a.

In [22]:
# Método de Newton Original com derivada numérica simples
def newton_original_derivada_numerica(x0, nu, mu, tol=1e-3, max_iter=100, h=1e-3):

    x = x0
    for i in range(max_iter):
        fx = f(x, nu, mu)
        fpx = derivada_numerica(f, x, nu, mu, h)  # Usando derivada simples
        if fpx == 0:  # Evita divisão por zero
            print("Derivada zero. Método não converge.")
            return None, None
        x_next = x - fx / fpx
        if abs(x_next - x) < tol:  # Critério de convergência
            return x_next, i + 1
        x = x_next
    print("Máximo de iterações atingido.")
    return x, max_iter


In [23]:
# Função para calcular o erro relativo
def erro_relativo(real, aproximado):
    if real is None or aproximado is None:
        return None
    return abs((real - aproximado) / real) if real != 0 else None

# Função para verificar se houve falha significativa
def falha_significativa(erro):
    return erro > 0.003  # Se o erro for maior que 0,3%, indica falha significativa

#Testando os resultados usando como padrão um equipamento v = 1, µ = 1, r0 = 0,5, λ = 0,05 e ϵ = 0,001.

In [24]:
# Definindo parâmetros
v = 1  # Velocidade
mu = 1  # Parâmetro μ
r0 = 0.5  # Valor inicial r0
lambda_valor = 0.05  # Valor de λ
epsilon = 0.001  # Precisão (tolerância)

In [25]:
# Resultados dos itens
resultados = []

In [37]:
# Adicionando o Risco de Produção ao DataFrame
# Função para garantir que o risco de produção não seja NaN
def garantir_risco_valid(risco):
    return risco if risco is not None and not np.isnan(risco) else 0.0  # Substituir NaN por 0.0 ou outro valor padrão

# Método de Newton Original
tempo_inicial = time.time()
raiz_original, iteracoes_original = newton_original(r0, v, mu, tol=epsilon)
tempo_original = time.time() - tempo_inicial

# Método de Newton com Fator de Limitação
tempo_inicial = time.time()
raiz_com_fl, iteracoes_com_fl = newton_com_fl(r0, v, mu, lambda_valor, tol=epsilon)
tempo_com_fl = time.time() - tempo_inicial

# Método de Newton Original com Derivada Numérica
tempo_inicial = time.time()
raiz_original_num, iteracoes_original_num = newton_original_derivada_numerica(r0, v, mu, tol=epsilon)
tempo_original_num = time.time() - tempo_inicial

# Calcular erro relativo
erro_original = erro_relativo(raiz_com_fl, raiz_original)
erro_com_fl = erro_relativo(raiz_original, raiz_com_fl)
erro_original_num = erro_relativo(raiz_com_fl, raiz_original_num)

# Verificar falha significativa
falha_original = falha_significativa(erro_original)
falha_com_fl = falha_significativa(erro_com_fl)
falha_original_num = falha_significativa(erro_original_num)

# Armazenar os dados dos resultados, incluindo Risco de Produção (Raiz)
resultados.append({
    "λ": lambda_valor,
    "v": v,
    "µ": mu,
    "Método": "Newton Original",
    "Raiz": raiz_original,
    "Risco de Produção": raiz_original,  # Adicionando Risco de Produção
    "Iterações": iteracoes_original,
    "Tempo (s)": tempo_original,
    "Erro Relativo": erro_original,
    "Falha Significativa": "Sim" if falha_original else "Não",
    "Derivada Numérica": "Não"
})

resultados.append({
    "λ": lambda_valor,
    "v": v,
    "µ": mu,
    "Método": "Newton Original com Derivada Numérica",
    "Raiz": raiz_original_num,
    "Risco de Produção": raiz_original_num,  # Adicionando Risco de Produção
    "Iterações": iteracoes_original_num,
    "Tempo (s)": tempo_original_num,
    "Erro Relativo": erro_original_num,
    "Falha Significativa": "Sim" if falha_original_num else "Não",
    "Derivada Numérica": "Sim"
})

resultados.append({
    "λ": lambda_valor,
    "v": v,
    "µ": mu,
    "Método": "Newton com FL",
    "Raiz": raiz_com_fl,
    "Risco de Produção": raiz_com_fl,  # Adicionando Risco de Produção
    "Iterações": iteracoes_com_fl,
    "Tempo (s)": tempo_com_fl,
    "Erro Relativo": erro_com_fl,
    "Falha Significativa": "Sim" if falha_com_fl else "Não",
    "Derivada Numérica": "Não"
})

# Criar um DataFrame com os resultados
df_resultados = pd.DataFrame(resultados)


#Fornecer um quadro resposta, com risco de produção r calculado para cada método dado (comparando resposta, acurácia (erro), tempo, número de iterações etc).
#Fornecer um quadro comparativo, com todos os dados para cada método dado (comparando resposta,acurácia (erro), tempo, número de iterações etc).


In [41]:
# Exibindo os resultados

# Criar um DataFrame para exibir o quadro
df_resposta = pd.DataFrame(resultados)

# Exibir o quadro
df_resposta


,λ,v,µ,Método,Raiz,Iterações,Tempo (s),Erro Relativo,Falha Significativa,Derivada Numérica,Risco de Produção
0,0.05,1,1,Newton Original,0.337609,3,0.000134,5.734025e-10,Não,Não,NaN
1,0.05,1,1,Newton Original com Derivada Numérica,0.337609,3,0.000076,0.000000e+00,Não,Sim,NaN
2,0.05,1,1,Newton com FL,0.337609,3,0.000071,5.734025e-10,Não,Sim,NaN
3,0.05,1,1,Newton Original,0.337609,3,0.000215,5.734025e-10,Não,Não,NaN
4,0.05,1,1,Newton Original com Derivada Numérica,0.337609,3,0.000150,0.000000e+00,Não,Sim,NaN
5,0.05,1,1,Newton com FL,0.337609,3,0.000128,5.734025e-10,Não,Sim,NaN
6,0.05,1,1,Newton Original,0.337609,3,0.000126,5.734025e-10,Não,Não,0.337609
7,0.05,1,1,Newton Original com Derivada Numérica,0.337609,3,0.000069,0.000000e+00,Não,Sim,0.337609
8,0.05,1,1,Newton com FL,0.337609,3,0.000084,5.734025e-10,Não,Não,0.337609
9,0.05,1,1,Newton Original,0.337609,3,0.000151,5.734025e-10,Não,Não,0.337609


#Analisar o efeito da variação do valor de v e µ para cada método considerado (comparando resposta, acurácia (erro), tempo, número de iterações etc).

In [42]:
# Função para organizar os resultados em um DataFrame
def analisar_variacao(v_valores, mu_valores, lambda_valores, epsilon=1e-3):
    resultados = []
    for lam in lambda_valores:
        for v in v_valores:
            for mu in mu_valores:
                # Métodos de Newton
                tempo_inicial = time.time()
                raiz_original, iteracoes_original = newton_original(0.5, v, mu, tol=epsilon)
                tempo_original = time.time() - tempo_inicial

                tempo_inicial = time.time()
                raiz_com_fl, iteracoes_com_fl = newton_com_fl(0.5, v, mu, lam, tol=epsilon)
                tempo_com_fl = time.time() - tempo_inicial

                tempo_inicial = time.time()
                raiz_com_derivada, iteracoes_com_derivada = newton_original_derivada_numerica(0.5, v, mu, tol=epsilon)
                tempo_com_derivada = time.time() - tempo_inicial

                # Calcular erro relativo
                erro_original = erro_relativo(raiz_com_fl, raiz_original)
                erro_com_fl = erro_relativo(raiz_original, raiz_com_fl)
                erro_com_derivada = erro_relativo(raiz_original, raiz_com_derivada)

                # Armazenar os resultados
                resultados.append({
                    "λ": lam,
                    "v": v,
                    "µ": mu,
                    "Método": "Newton Original",
                    "Raiz": raiz_original,
                    "Iterações": iteracoes_original,
                    "Tempo (s)": tempo_original,
                    "Erro Relativo": erro_original
                })

                resultados.append({
                    "λ": lam,
                    "v": v,
                    "µ": mu,
                    "Método": "Newton com FL",
                    "Raiz": raiz_com_fl,
                    "Iterações": iteracoes_com_fl,
                    "Tempo (s)": tempo_com_fl,
                    "Erro Relativo": erro_com_fl
                })

                resultados.append({
                    "λ": lam,
                    "v": v,
                    "µ": mu,
                    "Método": "Newton com Derivada Numérica",
                    "Raiz": raiz_com_derivada,
                    "Iterações": iteracoes_com_derivada,
                    "Tempo (s)": tempo_com_derivada,
                    "Erro Relativo": erro_com_derivada
                })

    # Criar DataFrame com os resultados
    return pd.DataFrame(resultados)

In [44]:
# Exemplos de valores para os parâmetros
lambda_valores = [0.05, 0.1, 0.2]
v_valores = [1, 2, 3]
mu_valores = [1, 2, 3]
#os valores para 𝜆, 𝑣v e 𝜇 que foram apresentados são exemplos aleatórios e são apenas para ilustrar como a variação desses parâmetros pode ser analisada.

In [48]:
# Obter os resultados
df_resultados = analisar_variacao(v_valores, mu_valores, lambda_valores, epsilon=1e-3)

# Exibir os resultados
print("\nQuadro comparativo de resultados:")
df_resultados



Quadro comparativo de resultados:


,λ,v,µ,Método,Raiz,Iterações,Tempo (s),Erro Relativo
0,0.05,1,1,Newton Original,0.337609,3,0.000020,5.734025e-10
1,0.05,1,1,Newton com FL,0.337609,3,0.000013,5.734025e-10
2,0.05,1,1,Newton com Derivada Numérica,0.337609,3,0.000010,5.734025e-10
3,0.05,1,2,Newton Original,0.166925,3,0.000016,2.244018e-10
4,0.05,1,2,Newton com FL,0.166925,3,0.000009,2.244018e-10
...,...,...,...,...,...,...,...,...
76,0.20,3,2,Newton com FL,0.167449,3,0.000008,2.278825e-08
77,0.20,3,2,Newton com Derivada Numérica,0.167449,3,0.000009,2.278825e-08
78,0.20,3,3,Newton Original,0.111264,3,0.000006,4.368602e-09
79,0.20,3,3,Newton com FL,0.111264,3,0.000009,4.368602e-09


In [49]:
# Agregar e analisar os resultados
resumo = df_resultados.groupby("Método").agg(
    Média_Raiz=("Raiz", "mean"),
    Média_Iterações=("Iterações", "mean"),
    Média_Tempo=("Tempo (s)", "mean"),
    Média_Erro=("Erro Relativo", "mean")
)

print("\nResumo dos resultados por método:")
resumo


Resumo dos resultados por método:


,Média_Raiz,Média_Iterações,Média_Tempo,Média_Erro
Método,,,,
Newton Original,0.206927,3.0,0.000010,1.399670e-08
Newton com Derivada Numérica,0.206927,3.0,0.000008,1.399670e-08
Newton com FL,0.206927,3.0,0.000009,1.399670e-08
